# **Fact checking, Neural Languange Inference (NLI)**

**Authors**: Giacomo Berselli, Marco Cucè, Riccardo De Matteo

In [ ]:
# to print all output for a cell instead of only last one 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

### 1. Libraries and Imports 

In [ ]:
import os
import requests
import zipfile
import random
import string 

import torch

import numpy as np
import pandas as pd

import gensim
import gensim.downloader as gloader

import time 

from collections import OrderedDict, namedtuple

# Fix data seed to achieve reproducible results
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [ ]:
print("Current work directory: {}".format(os.getcwd())) #print the current working directory 

data_folder = os.path.join(os.getcwd(),"data") # directory containing the notebook

if not os.path.exists(data_folder):   #create folder where all data will be stored 
    os.makedirs(data_folder)

### 2. Data handling

First thing first, we download the raw dataset, unzip it and store the csv document of each split in the dataset folder.

In [ ]:
raw_dataset_path = os.path.join(data_folder,'raw_dataset')   #path of the raw dataset as downloaded 

def save_response_content(response, destination):    
    CHUNK_SIZE =32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks                
                f.write(chunk)

def download_data(data_folder):
    zip_dataset_path = os.path.join(raw_dataset_path,'fever_data.zip')    
    data_url_id ="1wArZhF9_SHW17WKNGeLmX-QTYw9Zscl1"    
    url ="https://docs.google.com/uc?export=download"

    if not os.path.exists(raw_dataset_path):        
        os.makedirs(raw_dataset_path)

    if not os.path.exists(zip_dataset_path):
        print("Downloading FEVER data splits...")
        with requests.Session() as current_session:           
            response = current_session.get(url, params={'id': data_url_id}, stream=True)

        save_response_content(response, zip_dataset_path)
        print("Download completed!")

        print("Extracting dataset...")
        with zipfile.ZipFile(zip_dataset_path) as loaded_zip:            
            loaded_zip.extractall(raw_dataset_path)
        print("Extraction completed!")

download_data(data_folder)

Now that we have the csv files of the train, val and test splits, we encode all three as a unique pandas Dataframe to be able to better inspect it and manipulate it as a whole.
The Dataframe `df` is structured as follows: 
- `claim`: the fact to verify 
- `evidence`: one of the possibly multiple sentences in the dataset which supports or refutes the `claim`
- `id`: number associated to the fact to verify (different rows can have the same `id`)
- `label`: wether the evidence REFUTES or SUPPORTS the claim
- `split`: the split to which one claim belongs (train, val, test)


In [ ]:
#encode the entire dataset in a pandas dataframe and add the split column
def encode_dataset(): 

    df = pd.DataFrame()
    for split in ['train','val','test']:
        split_path = os.path.join(raw_dataset_path,f"{split}_pairs.csv")
        split_df = pd.read_csv(split_path,index_col=0)
        split_df['split'] = split

        df = df.append(split_df,ignore_index=True,)

    df.columns = df.columns.str.lower()
    df = df.reset_index(drop=True)

    return df 

df = encode_dataset()

Let's inspect the newly created dataset 

In [ ]:
df.head()
print('The splits present in the dataframe are:',df['split'].unique())
print('Unique labels in the dataset:',df['label'].unique())

From the above results we can see that the dataset has been structured correctly.\
Now we print some values to check the dimensions of the different splits and to retrive useful informations.

In [ ]:
print('Dataframe shape:', df.shape)
print('Number of example in train:',len(df[df['split']=='train']))
print('Number of example in val:',len(df[df['split']=='val']))
print('Number of example in test:',len(df[df['split']=='test']))

The number of claims in the training split of the dataset is clearly much higher than that of val and test splits.

The dataset should probably undergo some preprocessing before it can be used to train our model. Even if this was already noticeable from the few examples taken from the dataframe that we printed above, let's now show an examples of an evidence to make more evident the work that we will need to do.

In [ ]:
print(list(df.sample(1)['evidence']))

### 3. Text preprocessing 

BOH FORSE QUI TROVARE UN MODO PER VEDERE COSA ANDREBBE PULITO DAL DATASET 

Both claims and evidence contain a lot of unwanted text: punctuation, symbols, meta-characters, foreign words, tags, ecc. For some reason claims are much cleaner that evidences. Nonetheless we will preprocess both of them, to end up with a more manageable and digestible text. Especially since all the unwanted text do not contribute to the general meaning of each sentence, which is what we are interested in.
Our preprocessing pipeling will:
- drop everything before the first '\t' (every evidence seems to start with a number followed by '\t')
- delete all unnecessary spaces; only one space between each word will be left `QUESTO COMPRENDE \n \t \s ?` 
- remove all tabs and newlines characters (there are many '\t' in the dataset)  `???? MI SA CHE NON SERVE`
- remove the rounded parenthesis (-LRB- and -RRB-)
- drop words inside square brackets (everything that falls between -LSB- and -RSB-)
- delete all words that contains non-english/non-numerical characters  (there are some greek letters for instance)
- remove 's `E COSE SIMILI DA DEFINIRE O MAGARI NO`
- drop everything after the last dot character (after that there are often some other words similar to tags which may be image descriptions or hyperlinks)
- remove punctuation
- set everything to lowercase
- convert string in list of words 

`CAMBIARE L'ORDINE e AGGIUNGERNE ALTRE (la pipeline che c'è adesso è solo per andare avanti, possibili altre cose con nltk tipo stopwords, stemming, ecc `

In [ ]:
import re 

def preprocess_pipeline(sentence:str):
    
    #drop everything before the first '\t' 
    sentence = sentence[sentence.find('\t')+1:]

    #drop everything after the last period
    period_idx = sentence.rfind('.')
    if period_idx!= -1:
        sentence = sentence[:period_idx]

    #remove all rounded parenthesis 
    sentence = sentence.replace('-LRB-','').replace('-RRB-','')

    #remove words inside square brackets
    sentence = re.sub("-LSB.*?-RSB-","",sentence)

    sentence = sentence.replace('-LSB-','').replace('-RSB-','')

    #remove all punctuation
    sentence = sentence.translate(str.maketrans(dict.fromkeys(string.punctuation,' ')))

    #subsitute the charachten ˈ with a space 
    sentence = sentence.replace('ˈ',' ')

    #put everything to lowercase
    sentence = sentence.lower()

    #remove all unnecessary spaces and return a list of words
    sentence = sentence.split()

    return sentence

To test our preprocessing pipeline we will apply it to an example in the dataset that we have identified to be a pretty tough one in terms of amount of cleanup necessary.

In [ ]:
#retrive from the dataset the 13th example. It is one about Greece in which the text is pretty messy 
original_claim = df.loc[0,'claim']
original_evidence = df.loc[0,'evidence']

processed_claim = preprocess_pipeline(original_claim)
processed_evidence = preprocess_pipeline(original_evidence)

print('Original claim:',original_claim)
print('Processed claim:',processed_claim)
print('\nOriginal evidence:',original_evidence)
print('Processed evidence:',processed_evidence)


As we can see, the final results relative to both the claim and evidence after the preprocessing are satisfacory. For this reason we are now going to apply the preprocessing function to the entire dataset encoded as a Dataframe.

In [ ]:
df['claim'] = df['claim'].apply(preprocess_pipeline)
df['evidence'] = df['evidence'].apply(preprocess_pipeline)

df.head(10)

### 4. Vocabulary

Next, we have to build the dictionaries that will be used for the numerical tokenization of the dataset and for the generation of the embedding matrix.

The function `build_vocab` takes in input the list of unique words in the whole dataset and creates:
- `word2int`: dictionary which associates each word with an integer.
- `int2word`: dictionary which associates each integer with the relative word.

These two dictionaries constitute a bijective mapping between words and indexes in the dataset.

In [ ]:
Vocab = namedtuple('Vocabulary',['word2int','int2word','unique_words'])

def build_vocab(unique_words : list[str]): 
    """
        Builds the dictionaries word2int, int2word and put them in the Vocabulary
    """
    word2int = OrderedDict()
    int2word = OrderedDict()

    for i, word in enumerate(unique_words):
        word2int[word] = i+1           #plus 1 since the 0 will be used as tag token 
        int2word[i+1] = word
    
    return Vocab(word2int,int2word,unique_words)

The function `build_vocab` needs in input the list of all the unique words in the dataset, so we're now going to retrive it from the dataset to be able to build the dictionaries. 

In [ ]:
unique_words_claim = df['claim'].explode().unique().tolist()  
unique_words_evidence = df['evidence'].explode().unique().tolist()

print('the number of unique words belonging to claims is:', len(unique_words_claim))
print('the number of unique words belonging to evidences is:', len(unique_words_evidence))

unique_words = set(unique_words_evidence + unique_words_claim)
print('the number of unique words in the entire dataset is:', len(unique_words))


In [ ]:
vocab = build_vocab(unique_words)

Now that we have the vocabulary which contains the mapping between word and index we can 'numberise' the dataset. In particular we will add to the Dataframe 3 columns:
- `idx_claim`: same as `claim` but with each word substituted by its index.
- `idx_evidence`: same as `evidence` but with each word substituted by its index.
- `idx_label`: label encoding as a unique integer (0 or 1 in this case)

In [ ]:
def build_indexed_dataframe(df: pd.DataFrame):

    df['idx_claim'] = df.claim.apply(lambda x:list(map(vocab.word2int.get,x)))
    df['idx_evidence'] = df.evidence.apply(lambda x:list(map(vocab.word2int.get,x)))

    df['label'] = df.label.astype('category')   #convert the label column into category dtype
    df['idx_label'] = df.label.cat.codes        #assign unique integer to each category

    return df 

def check_dataframe_numberization(df,vocab):

    """
       Checks if the numberized dataframe will lead to the normal dataframe usind the reverse mapping 
    """

    claims = df['claim']
    evidences = df['evidence']

    idx_to_claims = df.idx_claim.apply(lambda x:list(map(vocab.int2word.get,x)))
    idx_to_evidences = df.idx_evidence.apply(lambda x:list(map(vocab.int2word.get,x)))

    if claims.equals(idx_to_claims) and evidences.equals(idx_to_evidences):
        print('All right with dataset numberization')
    else:
        print('There are problems with Dataset numberization')

df = build_indexed_dataframe(df)

check_dataframe_numberization(df,vocab)

Since the operation was successful, let's have a look at the numebrized dataframe.

In [ ]:
df.head()

### 5. Data Loaders 

In order generate mini-batches for each split to be passed to the network we leveraged a `torchtext` utility, such as `BucketIterator`. It ensures that each mini-batch is composed of sequences of nearly the same length (depending on the chosen batch size), in order to add the minimum padding possible to each Tensor. In order to do so, we needed to create a Pytorch Dataset since this is what is requested by the BucketIterator.\
The problem is now how to define the length of the input to the model (which is used to create buckets of similar-lenghts senquences), since for this task we are dealing with multiple inputs (claim and evidence). 


In [ ]:
claim_lengths = df.claim.apply(len)
evidence_lengths = df.evidence.apply(len)
print('average length of a claim sentence:',claim_lengths.mean())
print('average length of a evidence sentence:',evidence_lengths.mean())
print('max difference in length of claim sentences:',claim_lengths.max() - claim_lengths.min())
print('max difference in length of evidence sentences:',evidence_lengths.max() - evidence_lengths.min())

Based on QUALCOSA we decided QUALCOSA

In [ ]:
from torchtext.legacy.data import BucketIterator
from torch.utils.data import Dataset

class DataframeDataset(Dataset):

    def __init__(self, dataframe: pd.DataFrame):
        self.claims = dataframe['idx_claim']      #column of numberized claims 
        self.evidences = dataframe['idx_evidence']   #column of numberized evidences 
        self.labels = dataframe['idx_label']       #column of categorical label 
        self.claim_ids = dataframe['id']          #column of claim ids 

    def __len__(self):
        return len(self.claims)

    def __getitem__(self, idx):
        return {'claim': self.claims[idx],
                'evidence': self.evidences[idx],
                'label': self.labels[idx],
                'claim_id': self.claim_ids[idx]}

def create_dataloaders(b_s : int, dataframe: pd.DataFrame):     #b_s = batch_size
    
    train_df = dataframe[dataframe['split'] == 'train'].reset_index(drop=True)      
    val_df = dataframe[dataframe['split'] == 'val'].reset_index(drop=True)
    test_df = dataframe[dataframe['split'] == 'test'].reset_index(drop=True)

    #create DataframeDataset objects for each split 
    train_dataset = DataframeDataset(train_df)
    val_dataset = DataframeDataset(val_df)
    test_dataset = DataframeDataset(test_df)


    # Group similar length text sequences together in batches and return an iterator for each split.
    train_dataloader,val_dataloader,test_dataloader = BucketIterator.splits((train_dataset,val_dataset,test_dataset),
                                                        batch_sizes=(b_s,b_s,b_s), sort_key=lambda x: (len(x['claim']),len(x['evidence'])), 
                                                        repeat=True, sort=False, shuffle=True, sort_within_batch=True)
    
    return train_dataloader,val_dataloader,test_dataloader 


In [ ]:
tr, vl, ts = create_dataloaders(16,df)

In [ ]:
tr.init_epoch()
tr.create_batches()

In [ ]:
tr.batches

In [ ]:
for batch in tr.batches:
    print([len(example['claim']) for example in batch])
    print([len(example['evidence']) for example in batch])

    print(list(map(vocab.int2word.get,batch[2]['claim'])))
    print(list(map(vocab.int2word.get,batch[2]['evidence'])))
    print(batch[2]['label'])
    break